<br>
<br>

<p style='color:red;'><b>본 실습 중 다뤄지는 코드의 경우 구글 번역 웹사이트의 정책 및 소스코드 변경에 따라 일부가 정상적으로 작동하지 않을 수 있습니다.</b></p>
<p style='color:red;'>구글 번역 웹사이트의 구조 변경으로 인해 코드가 실행되지 않을 시, 각 <b>태그의 XPath(XML Path Language)를 복사하여 덮어쓰기</b>할 경우 작동될 가능성이 높습니다.</p>

In [1]:
!pip install openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 KB 10.4 MB/s eta 0:00:00
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [2]:
!pip install selenium==4.1.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.8/958.8 KB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 KB 57.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 KB 12.3 MB/s eta 0:00:00
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [3]:
!pip install webdriver-manager==3.5.2

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


#### Selenium & Driver(for browser) 준비

- Selenium : http://selenium-python.readthedocs.io/ 에서 공식 문서 확인 가능
- ~~ChromeDriver 다운로드 후 작업 폴더에 위치시키기~~ (Webdriver-manager 라이브러리를 활용해 자동으로 다운로드 가능)

In [14]:
#from webdriver_manager.chrome import ChromeDriverManager # 자동으로 크롬드라이버(가상브라우저) 파일을 다운로드해주는 라이브러리
#from selenium.webdriver.chrome.service import Service # 다운로드된 크롬드라이버 파일을 연결하기 위해 활용

#from selenium import webdriver
#from selenium.webdriver.common.by import By
#from selenium.webdriver.common.keys import Keys

from bs4 import BeautifulSoup 
import time
import pandas as pd
import requests
from urllib.request import urlopen

import warnings
warnings.filterwarnings("ignore") # 불필요한 Warning 메시지를 꺼줍니다.

<br>
<br>

## 1. Selenium 둘러보기

In [5]:
df = pd.read_excel('result_220202_1834.xlsx')
df.head()

,Title,Date,Article,URL,PressCompany
0,"인천테크노파크, 중소기업 빅데이터 지원사업 '우수' 등급",2022.02.02. 오전 10:09,[인천=뉴시스] 이루비 기자 = 인천테크노파크 로고. (이미지=인천테크노파크 제공...,https://news.naver.com/main/read.naver?mode=LS...,뉴시스
1,"식약처, 식·의약 데이터 분석 플랫폼 구축",2022.02.01. 오전 8:28,[KBS 청주]식품의약품안전처는 오는 4월부터 식·의약 데이터를 모은 분석 플랫폼을...,https://news.naver.com/main/read.naver?mode=LS...,KBS
2,"2촌까지 가능한 유전자 분석기술 도입될까…""실종자 가족 희망""",2022.02.01. 오전 7:16,유전자[게티이미지뱅크 제공] (서울=연합뉴스) 이정현 기자 = 실종자 확인 시...,https://news.naver.com/main/read.naver?mode=LS...,연합뉴스
3,"길영준 휴이노 대표 ""2월 원격 모니터링 시장 첫발…최대 규모 임상 데이터 확보 목표""",2022.02.01. 오전 10:01,이달부터 심전도 장기검사 보험 신설휴이노 최대 4000명 규모 임상 계획홀터 검사 ...,https://news.naver.com/main/read.naver?mode=LS...,서울경제
4,“빅데이터 잘 쓰면 매출 2배도 가능해요”,2022.01.28. 오전 11:18,"KT, 소상공인 빅데이터 성공 사례 공유상권분석 플랫폼 ‘잘나가게’ 활용법 오픈KT...",https://news.naver.com/main/read.naver?mode=LS...,헤럴드경제


In [7]:
# 활용 중인 개발환경에 따라 Selenium 4.1.0 버전 혹은 Webdriver-manager 3.5.2 버전이 문제를 일으킬 수 있습니다.
# 그러한 경우에는 selenium 버전을 낮추고 아래와 같은 절차를 통해 예전 방식으로 가상브라우저를 띄울 수 있습니다.


# 1. cmd를 관리자권한으로 실행 후, 아래 명령어를 입력하여 Selenium 버전을 다운그레이드합니다.

!pip install selenium==3.11.0


# 2. 아래 코드를 Jupyter notebook에 입력해 가상브라우저를 띄웁니다. ('(driver) chromedriver.exe' 파일이 ipynb 파일 옆에 있어야 합니다.)

driver = webdriver.Chrome(executable_path='\work\(driver) chromedriver.exe') 


# 3-1. 만약 위 코드 실행 시 [ WebDriverException (cannot find Chrome binary) ] 관련 에러가 발생하면,
#    실제 크롬 브라우저가 설치된 경로를 찾으셔서 아래와 같이 Chrome의 option값을 세팅해주세요. (options.binary_location 변수)

from selenium.webdriver.chrome.options import Options
options = Options()
options.binary_location = "C:\\Program Files\\Google\\Chrome\\Application\\chrome.exe" # 실제 크롬 브라우저 설치 경로를 찾아 변경 후 적용
driver = webdriver.Chrome(executable_path='\work\(driver) chromedriver.exe', chrome_options=options) # webdriver 실행 시마다 동일하게 작성


# 3-2. 혹은 위와 같은 코드 상의 변경 대신 https://chromedriver.chromium.org/downloads 에서 
#      현재 설치된 크롬 브라우저의 버전과 동일한 버전의 ChromeDriver를 다운로드 받아,
#      코드를 실행하고 있는 ipynb 파일과 동일한 폴더 내의 '(driver) chromedriver.exe' 파일을 대체해주셔도 문제가 해결될 수 있습니다.


# * MacOS의 경우 참고 : Mac OS 환경에서 Selenium Driver Path 설정하기 : https://j.mp/3reMri7

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


WebDriverException: Message: '\work\(driver) chromedriver.exe' executable needs to be in PATH. Please see https://chromedriver.chromium.org/home


In [8]:
# 자동으로 크롬드라이버(가상브라우저) 파일을 다운로드 후 세팅
service = Service(executable_path=ChromeDriverManager().install()) 

# 세팅된 크롬드라이버를 연결해 가상브라우저 실행
driver = webdriver.Chrome(service=service)



====== WebDriver manager ======
Could not get version for google-chrome with the any command: google-chrome --version || google-chrome-stable --version
Current google-chrome version is UNKNOWN
Get LATEST chromedriver version for UNKNOWN google-chrome
There is no [linux64] chromedriver for browser  in cache
Trying to download new driver from https://chromedriver.storage.googleapis.com/111.0.5563.64/chromedriver_linux64.zip
Driver has been saved in cache [/root/.wdm/drivers/chromedriver/linux64/111.0.5563.64]


WebDriverException: Message: Service /root/.wdm/drivers/chromedriver/linux64/111.0.5563.64/chromedriver unexpectedly exited. Status code was: 127


In [ ]:
# driver.maximize_window() # 가상 브라우저를 최대화하고 싶을 때 활용하세요

In [ ]:
translate_url = 'https://translate.google.co.kr/?sl=auto&tl=en&op=translate&hl=ko' 
# hl=ko : Korean & tl=en : English

driver.get(translate_url) # Get the page
print(driver.current_url)

<br>

## 문장 번역 & 크롤링

<p style='color:red;'>위 스크린샷 화면과 같이 개발자도구에서 원본 텍스트를 입력받는 태그를 찾은 후,</p>
<p><b style='color:red;'>개발자도구 영역에서 우클릭 -> "Copy" -> "Copy full XPath" 를 클릭해 XPath를 복사해옵니다. </b>(<a href='https://testmanager.tistory.com/121'>XPath란?</a>)</p> 

In [ ]:
origin_xpath = '/html/body/c-wiz/div/div[2]/c-wiz/div[2]/c-wiz/div[1]/div[2]/div[3]/c-wiz[1]/span/span/div/textarea'

driver.find_element_by_xpath(origin_xpath).send_keys('파이썬은 쉽습니다') # 기존 방식, XPath & Send_keys
# driver.find_element(By.XPATH, origin_xpath).send_keys('...') # 새로운 방식 (원할 경우 활용하세요)
# .send_keys('') / .clear() / .click() / .text

<p style='color:red;'>번역 결과 역시도 마찬가지로 위 스크린샷 화면과 같이 개발자도구에서 태그를 찾은 후,</p>
<p><b style='color:red;'>개발자도구 영역에서 우클릭 -> "Copy" -> "Copy full XPath" 를 클릭해 XPath를 복사해옵니다. </b></p> 

In [ ]:
translation_xpath = '/html/body/c-wiz/div/div[2]/c-wiz/div[2]/c-wiz/div[1]/div[2]/div[3]/c-wiz[2]/div[8]/div/div[1]/span[1]/span/span'

translated_contents = driver.find_element_by_xpath(translation_xpath) # XPath

print(translated_contents)
print(translated_contents.text) # text (not get_text())

<br>

## 기사글 번역 & 크롤링

In [ ]:
article = df['Article'][0]

origin_xpath = '/html/body/c-wiz/div/div[2]/c-wiz/div[2]/c-wiz/div[1]/div[2]/div[3]/c-wiz[1]/span/span/div/textarea'
driver.find_element_by_xpath(origin_xpath).clear() # 깔끔하게 비우기(clear)
driver.find_element_by_xpath(origin_xpath).send_keys(article)
time.sleep(3)

translation_xpath = '/html/body/c-wiz/div/div[2]/c-wiz/div[2]/c-wiz/div[1]/div[2]/div[3]/c-wiz[2]/div[8]/div/div[1]/span[1]/span/span'
translated_contents = driver.find_element_by_xpath(translation_xpath).text

print('기사글 [ {} ] 의 번역이 끝났습니다.'.format(df['Title'][0]))
print(translated_contents)

In [ ]:
driver.close()
driver.quit()

<br>
<br>

## 2. 기사 원문 전체 번역 (시간이 오래 걸립니다 & 중간에 IP가 막힐 수 있습니다)

In [ ]:
# eng_contents = []

# service = Service(executable_path=ChromeDriverManager().install()) 
# driver = webdriver.Chrome(service=service)

# translate_url = 'https://translate.google.co.kr/?sl=auto&tl=en&op=translate&hl=ko' 
# driver.get(translate_url) 
# print(driver.current_url)
# time.sleep(3)

# for row_index, row in df.iterrows():
#     origin_xpath = '/html/body/c-wiz/div/div[2]/c-wiz/div[2]/c-wiz/div[1]/div[2]/div[3]/c-wiz[1]/span/span/div/textarea'
#     driver.find_element_by_xpath(origin_xpath).clear()
#     driver.find_element_by_xpath(origin_xpath).send_keys(df['Article'][row_index])
#     time.sleep(3)

#     translation_xpath = '/html/body/c-wiz/div/div[2]/c-wiz/div[2]/c-wiz/div[1]/div[2]/div[3]/c-wiz[2]/div[8]/div/div[1]/span[1]/span/span'
#     translated_contents = driver.find_element_by_xpath(translation_xpath).text
#     eng_contents.append(translated_contents)
#     print('기사글 [ {} ] 의 번역이 끝났습니다.'.format(df['Title'][row_index]))

# print('전체 contents 번역이 끝났습니다!')

# driver.close()
# driver.quit()


# df['Translated_article'] = eng_contents
# df.to_excel('translation_result.xlsx', index=False, encoding='utf-8')

# print('crawling_result.xlsx 파일로 전체 저장이 완료되었습니다!')

<br>
<br>

## 3. 한글 word_dict에서 상위 항목만 단어로 가져와 번역

In [ ]:
from collections import Counter
from konlpy.tag import Okt

articles = df['Article'].tolist()
articles = ''.join(articles)

tokenizer = Okt()
raw_pos_tagged = tokenizer.pos(articles, norm=True, stem=True)

del_list = ['하다', '있다', '되다', '이다', '돼다', '않다', '그렇다', '아니다', '이렇다', '그렇다', '어떻다'] 
word_cleaned = []
for word in raw_pos_tagged:
    if not word[1] in ["Josa", "Eomi", "Punctuation", "Foreign"]: # Foreign == ”, “ 와 같이 제외되어야할 항목들
        if (len(word[0]) != 1) & (word[0] not in del_list): # 한 글자로 이뤄진 단어들을 제외 & 원치 않는 단어들을 제외
            word_cleaned.append(word[0])

word_counted = Counter(word_cleaned)
word_dic = dict(word_counted)
        
sorted_word_dic = sorted(word_dic.items(), key=lambda x:x[1], reverse=True)
# sorted_word_dic

In [ ]:
# 빈도 순으로 상위 100개에 해당하는 단어까지만 추려내어 번역을 진행합니다. (늘릴 수 있습니다.)

translation_target = sorted_word_dic[:100] 
translation_target[-1] # 상위 100번째에 해당하는 단어의 빈도수를 확인합니다. (가장 마지막 item을 호출하려면?)

In [ ]:
translation_target = {}
for key in word_dic:
    if word_dic[key] >= 24: # 위에서 확인한 빈도수를 기반으로 번역 대상 단어를 선정합니다.
        translation_target[key] = word_dic[key]

In [ ]:
print(translation_target)

In [ ]:
translation_result = {}

service = Service(executable_path=ChromeDriverManager().install()) 
driver = webdriver.Chrome(service=service)

translate_url = 'https://translate.google.co.kr/?sl=auto&tl=en&op=translate&hl=ko'
driver.get(translate_url) 
print(driver.current_url)
time.sleep(3)

In [ ]:
# 만약 바로 아래 Cell 실행 시 [ NoSuchElementException (no such element: Unable to locate element) ] 에러가 발생하면,
# 네트워크 속도 등의 이슈로 인해 3초라는 사전에 설정된 시간 동안 기다렸으나 번역 결과가 아직 생성되지 않은 것입니다. (HTML Tag가 로딩되지 않음)

# 혹은 [ 서로 다른 한글 형태소 값에 대한 번역 결과로 연속적으로 동일한 영문 단어가 print되는 현상 ]도 동일한 원인에 기인합니다.
# 따라서 네트워크 속도에 따라 아래 time.sleep(3)에서 3을 5~7으로 증가시켜주면 보다 안정적으로 결과를 받아올 수 있습니다.

In [ ]:
for key in translation_target: # 상위 100번째 빈도수에 해당하는 단어까지 담겨있는 dict
    
    origin_xpath = '/html/body/c-wiz/div/div[2]/c-wiz/div[2]/c-wiz/div[1]/div[2]/div[3]/c-wiz[1]/span/span/div/textarea'
    driver.find_element_by_xpath(origin_xpath).clear()
    driver.find_element_by_xpath(origin_xpath).send_keys(key)
    time.sleep(3) # 네트워크의 속도에 따라 잠깐씩 쉬어주면서 진행합니다.

    translated_xpath = '/html/body/c-wiz/div/div[2]/c-wiz/div[2]/c-wiz/div[1]/div[2]/div[3]/c-wiz[2]/div[8]/div/div[1]/span[1]/span/span'
    translated_contents = driver.find_element_by_xpath(translated_xpath).text
    
    translation_result[translated_contents] = translation_target[key] # 번역이 완료된 단어 dict의 value로 기존 dict의 value를 꽂아줍니다.
    print('단어 {}의 번역 완료 : {}'.format(key, translated_contents))

print('전체 번역이 끝났습니다!')

driver.close()
driver.quit()

In [ ]:
translation_result['It'] # 구글 번역 서비스의 성능 향상에 따라 정상적으로 번역이 진행될 시 해당 key는 들어있지 않을 수 있습니다.

In [ ]:
# 구글 번역 서비스의 성능 향상에 따라 정상적으로 번역이 진행될 시 아래 코드는 필요하지 않을 수 있습니다. ("AI" & "가다" 번역 결과 확인)

translation_result['AI'] = translation_result['It'] # AI
translation_result['go'] = translation_result['good morning my love'] # 가다

del translation_result['It']
del translation_result['good morning my love']

In [ ]:
from PIL import Image 
from wordcloud import WordCloud, ImageColorGenerator # Image 로부터 Color 를 생성(Generate)해내는 객체입니다.
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

python_coloring = np.array(Image.open("python_mask.jpg"))
image_colors = ImageColorGenerator(python_coloring)

word_cloud = WordCloud(font_path="C:/Windows/Fonts/malgun.ttf", # font_path="C:/Windows/Fonts/NanumSquareB.ttf"
                       width=2000, height=1000,  
                       mask=python_coloring, 
                       background_color='white').generate_from_frequencies(translation_result)

plt.figure(figsize=(15,15))
plt.imshow(word_cloud.recolor(color_func=image_colors), interpolation='bilinear') # 다시(re) 색칠하기
# plt.imshow(word_cloud.recolor(colormap='Blues'), interpolation='bilinear') # Matplotlib colormap 활용 (http://j.mp/32UXOQ6)
plt.axis("off")
plt.tight_layout(pad=0)
plt.show()

<br>

## (Appendix 1) 네이버 파파고를 활용한 번역 (find_element_by_id 활용)

In [ ]:
service = Service(executable_path=ChromeDriverManager().install()) 
driver = webdriver.Chrome(service=service)

In [ ]:
translate_url = 'https://papago.naver.com/?sk=ko&tk=en'
# sk=ko : Korean & tk=en : English

driver.get(translate_url) 
print(driver.current_url)

In [ ]:
driver.find_element_by_id('txtSource').send_keys('파이썬은 쉽습니다.') # Send_keys

In [ ]:
translated_contents = driver.find_element_by_id('txtTarget')
print(translated_contents.text)

driver.close()
driver.quit()

<p>파파고 번역 웹사이트의 경우 번역 키워드 입력 후 번역된 결과가 노출될 때까지의 간격이 긴 편입니다.</p>
<p>이를 일정한 시간(ex. 3초)으로 지정해 기다릴 경우 종종 <b>번역이 채 끝나지 않은 상태에서 잘못된 번역 결과를 가져오게 됩니다.</b></p>

<br>

<p>번역이 성공적으로 종료된 시점에만 눈 앞에 나타나는 어떠한 태그가 있을 경우, </p>
<p>우리는 <b style='color:red;'>이 태그가 눈 앞에 나타날 때까지 대기하여 (특정한 시간만큼 기다리는 대신) 번역 완료 시까지 유동적으로 기다릴 수 있습니다.</b> </p>

<br>

<p>위 스크린샷에서 <b style='color:red;'>번역이 완료될 경우 나타나는 발음(diction)이 담긴 태그("diction_~" class를 가진 p 태그)</b>가 바로 그러한 태그입니다.</p>
<p>실제로 <b>위 p 태그는 매 번역 키워드 입력 시마다 잠시 동안 사라졌다가 번역이 마쳐지면 다시 나타납니다.</b></p>
<p>우리는 아래 코드에서 <b>expected_conditions</b>를 활용해 이와 같은 <b style='color:red;'>특정한 태그의 로딩이 완료될 때까지 기다린 후</b> 번역 결과를 가져올 수 있습니다.</p>

<hr>

## 아래 cell 실행 후에는 가상브라우저를 화면 가장 앞에 띄워 활성화된 상태를 유지해주세요.

<br>

- 인터넷 환경 혹은 컴퓨터 환경(메모리 용량 등)에 따라 실행이 원활하지 않을 수 있습니다. (동일한 번역 결과값이 반복하여 출력됨)
- 이러한 경우 Zoom 등 메모리를 많이 차지하는 프로그램들을 종료하고 높은 인터넷 속도가 확보된 상태에서 다시 시도해보시길 권장드립니다.

<hr>

In [ ]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.by import By


translation_result_papago = {}

service = Service(executable_path=ChromeDriverManager().install()) 
driver = webdriver.Chrome(service=service)

translate_url = 'https://papago.naver.com/?sk=ko&tk=en'
driver.get(translate_url) 
print(driver.current_url)
time.sleep(3)
    
for key in translation_target: # 상위 100번째 빈도수에 해당하는 단어까지 담겨있는 dict
    
    driver.find_element_by_id('txtSource').clear()
    driver.find_element_by_id('txtSource').send_keys(key)
    time.sleep(3) # 네트워크의 속도에 따라 잠깐씩 쉬어주면서 진행합니다.

    wait = WebDriverWait(driver, timeout=10) # timeout : 지정한 조건의 충족까지 대기할 최대 시간 (만약 10초 내에 충족되지 않으면 에러가 발생)
    wait.until(expected_conditions.presence_of_element_located((By.CSS_SELECTOR, "#targetEditArea > p"))) # #은 id이고 >는 부모 태그 안의~ 라는 뜻이다.
    # 위 CSS Selector로 지정한 Tag의 존재가 포착될 때까지 대기 후 아래 코드를 실행 (가능한 조건 전체 목록 @ https://j.mp/3mCnc5G)
    
    translated_contents = driver.find_element_by_id('txtTarget').text
    
    translation_result_papago[translated_contents] = translation_target[key] # 번역이 완료된 단어 dict의 value로 기존 dict의 value를 꽂아줍니다.
    print('단어 {}의 번역 완료 : {}'.format(key, translated_contents))

print('전체 번역이 끝났습니다!')

driver.close()
driver.quit()

<br>

## (Appendix 2) 인터파크 투어 크롤링 예시

In [ ]:
print('http://tour.interpark.com/')

In [ ]:
service = Service(executable_path=ChromeDriverManager().install()) 
driver = webdriver.Chrome(service=service)

driver.get('http://tour.interpark.com/')
driver.current_url

In [ ]:
driver.find_element_by_id('SearchGNBText').send_keys('보라카이')
driver.find_element_by_class_name('search-btn').click()

In [ ]:
# driver.implicitly_wait(5)

In [ ]:
driver.page_source

In [ ]:
web_page = BeautifulSoup(driver.page_source, 'html.parser')
# web_page

In [ ]:
driver.find_element_by_class_name('moreBtn').click()
#이후에 find elements로 각 여행지의 링크를 저장한 뒤, request로 활용?

In [18]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'}
web = requests.get('https://search-travel.interpark.com/search?q=보라카이&cateCode=tourR', headers = headers).content #왜 크롤링이 안되는지..
requests.get()
web_page = BeautifulSoup(web, 'html.parser')
#web_page = BeautifulSoup(driver.page_source, 'html.parser')
print(web_page.contents)
products = web_page.find('ul', {'id':'boxList'})
print(products)
#.find_all('li', {'class':'boxItem'})

prd_title_list = []
prd_price_list = []
prd_date_start_list = []
prd_date_end_list = []
prd_img_src_list = []

for product in products:
    product_title = product.find('h5',{'class':'infoTitle'}) #상품 제목
    product_price = product.find('strong') #금액
    product_date = product.find_all('p', {'class':'info'})[1].split(':').strip().split('~') #출발가능기간
    product_date_start = product_date[0]
    product_date_end = product_date[1]
    product_img_src = product.find('img').attrs['src']

    prd_title_list.append(product_title)
    prd_price_list.append(product_price)
    prd_date_start_list.append(product_date_start)
    prd_date_end_list.append(product_date_end)
    prd_img_src_list.append(product_img_src)

df. pd.DataFrame({'product_title': prd_title_list,
                    'product_price' : prd_price_list,
                    'product_start' : prd_date_start_list,
                    'product_end' : prd_date_end_list,
                    'product_image' : prd_img_src_list
                    })
df.head()


['html', <html lang="ko"><head><title>인터파크 투어</title><meta charset="utf-8"/><meta content="width=device-width,initial-scale=1,shrink-to-fit=no" name="viewport"/><meta content="인터파크 투어" name="description"/><meta content="INTERPARK" name="author"/><meta content="IE=edge" http-equiv="X-UA-Compatible"/><meta content="telephone=no" name="format-detection"/><meta content="tour" name="service"/><meta content="standard" name="compatibility"/><meta content="1280" name="containerWidth"/><link href="//openimage.interpark.com/tourpark/tour/common/interparkn.ico" rel="shortcut icon"/><script charset="euc-kr" src="//tour.interpark.com/global/js/common.js" type="text/javascript"></script><script charset="euc-kr" src="//tour.interpark.com/global/js/gate.js" type="text/javascript"></script><script src="/scripts/jquery-1.12.4.min-4f252523d4af0b478c810c2547a63e19.js" type="text/javascript"></script><script src="/scripts/jquery.nstSlider.min-2302ae9581569f59631b2fd8b5f6d8c0.js" type="text/javascript"></sc

TypeError: 'NoneType' object is not iterable

In [ ]:
# 2페이지로 페이지 변경

driver.find_element_by_xpath('/html/body/div[3]/div/div[1]/div[2]/div[3]/div[2]/div[3]/ul/li[2]').click()

In [ ]:
# 1페이지로 페이지 변경

# driver.find_element_by_xpath('/html/body/div[3]/div/div[1]/div[2]/div[3]/div[2]/div[3]/ul/li[1]').click()

In [ ]:
driver.close()
driver.quit()

<br>

## (Appendix 3) Selenium - Headless mode 

In [ ]:
from selenium.webdriver.chrome.options import Options

options = Options()
options.headless = True

In [ ]:
service = Service(executable_path=ChromeDriverManager().install()) 
driver = webdriver.Chrome(service=service, options=options)

driver.get('http://tour.interpark.com/')
time.sleep(2)
print(driver.current_url)

driver.find_element_by_id('SearchGNBText').send_keys('보라카이')
driver.find_element_by_class_name('search-btn').click()
time.sleep(2)

driver.find_element_by_class_name('moreBtn').click()
time.sleep(2)

web_page = BeautifulSoup(driver.page_source, 'html.parser')
items = web_page.find('ul', {'id':'boxList'}).find_all('li', {'class':'boxItem'})
for item in items:
    print(item.find('h5', {'class':'infoTitle'}))

driver.close()
driver.quit()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=b844bbc7-67c4-4320-a052-bd9703f8a211' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>